# Agentic tool usage by local LLMs with LangChain
In this notebook I will create several simple examples of Agentic tool usage by local LLMs implemented with LangChain framework

## Some examples from LM-Studio docs

In [1]:
from openai import OpenAI

In [98]:
# check if the server is running and the model is available
! curl http://localhost:1234/v1/models

{
  "data": [
    {
      "id": "glm-4.5-air-mlx",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-30b-a3b-thinking-2507",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-coder-30b-a3b-instruct",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-30b-a3b-instruct-2507",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-32b",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "gemma-3-27b-it-qat",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "text-embedding-nomic-embed-text-v1.5",
      "object": "model",
      "owned_by": "organization_owner"
    }
  ],
  "object": "list"
}

In [ ]:
# Point to the local server
client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="qwen3-coder-30b-a3b-instruct",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Greetings, I'm Qwen, a language model from Alibaba Cloud's Tongyi Lab. \nI'm here to assist you with various questions and tasks.\nWith great knowledge and a helpful heart,\nI aim to make your day a bit more sparkly. \nWhether you need help with writing, coding, or just want to chat,\nI'm ready to lend a hand and make things better.\nSo feel free to ask, I'm here to serve,\nWith wit and wisdom, I'll keep you entertained.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [ ]:
# Connect to LM Studio
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Define a simple function
def say_hello(name: str) -> str:
    print(f"Hello, {name}!")

# Tell the AI about our function
tools = [
    {
        "type": "function",
        "function": {
            "name": "say_hello",
            "description": "Says hello to someone",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The person's name"
                    }
                },
                "required": ["name"]
            }
        }
    }
]

# Ask the AI to use our function
response = client.chat.completions.create(
    model="qwen3-coder-30b-a3b-instruct",
    messages=[{"role": "user", "content": "Can you say hello to Bob the Builder?"}],
    tools=tools
)

# Get the name the AI wants to use a tool to say hello to
# (Assumes the AI has requested a tool call and that tool call is say_hello)
tool_call = response.choices[0].message.tool_calls[0]
name = eval(tool_call.function.arguments)["name"]

# Actually call the say_hello function
say_hello(name) # Prints: Hello, Bob the Builder!



## LangChain for LLM agent tooling
LangChain for building agent tooling with your local LLM served by LM Studio via the OpenAI-compatible API. Here's a step-by-step guide:

__📊 Visualization of the model reasoning steps__

User Query: "What is 15*4+2?"
│
├─ Tool Descriptions Check:
│   ├─ "Useful for math problems" → MATCH ✅
│   └─ "Weather tool" → No match ❌
│
└─ Decision:
   ├─ Tool Use Priority: HIGH (matched tool exists)
      └─ Execute calculator
          Input: "15 * 4 + 2"
          Output: 62
   └─ Pre-training Knowledge: NOT USED

---

User Query: "What is photosynthesis?"
│
├─ Tool Descriptions Check:
│   ├─ "Math tool" → No match ❌
│   └─ "Science tool" → No match ❌ (if not defined)
│
└─ Decision:
   ├─ Tool Use Priority: LOW (no matches)
      └─ Skip tools
   └─ Pre-training Knowledge USED:
       Final Answer: "Photosynthesis is how plants..."


In [42]:
# separatly test the custom tool

def calculator(expression: str) -> float:
    expr = expression.strip('"\'')
    return eval(expr)

calculate("15 * 4 + 2")

62

### Method 1 - with LangChain Tool object

In [38]:
#from langchain.chat_models import ChatOpenAI# OLD (pre-v0.2)
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.llms import OpenAI

# LM Studio settings
BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

# Initialize the LLM
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",  # Replace with your actual model name
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

def calculate(expression: str) -> float:
    """Evaluate a mathematical expression."""
    # Remove surrounding quotes if present
    stripped_expr = expression.strip('"\'')
    return eval(stripped_expr)


# Wrap the function in LangChain's Tool format
tools = [
    Tool(
        name="calculator",
        func=calculate,
        description="Useful for when you need to solve math problems. Input should be a mathematical expression as a string, like '15 * 4 + 2'"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # Shows reasoning process
)

response = agent.run('''What is 15 * 4 + 2?''')
print(response)  # Expected: "62"




> Entering new AgentExecutor chain...

<think>The human is asking for the result of a simple mathematical expression: "15 * 4 + 2". This is straightforward arithmetic that I can compute using the calculator tool.

I need to provide a JSON blob with:
- action: "calculator" (since I need to perform a calculation)
- action_input: the mathematical expression as a string, which is "15 * 4 + 2"

Let me create this JSON blob:</think>Action:
```
{
  "action": "calculator",
  "action_input": "15 * 4 + 2"
}
```
Observation: 62
Thought:
<think>The calculator tool has returned a result of 62 for the expression "15 * 4 + 2". Let me verify this calculation:

15 * 4 = 60
60 + 2 = 62

The result is correct. Now I need to provide a final answer using the "Final Answer" action in the JSON blob format.</think>Action:
```
{
  "action": "Final Answer",
  "action_input": "The result of 15 * 4 + 2 is 62."
}
```

> Finished chain.
The result of 15 * 4 + 2 is 62.


### Method 2 - with LangChain @tool decorator

Your function's docstring becomes the __tool's description__.

When the agent decides to use a tool-> It calls calculator("input"). The LLM reads the docstring to understand:

What the tool does, What input format it expects, When to use it

In [39]:
#from langchain.chat_models import ChatOpenAI# OLD (pre-v0.2)
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain_core.tools import tool

# LM Studio settings
BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

# Initialize the LLM
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",  # Replace with your actual model name
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# Tool definition using decorator (this is the complete tool)
@tool
def calculator(expression: str) -> float:
    """Useful for when you need to solve math problems. Input should be a mathematical expression as a string, like '15 * 4 + 2'"""
    expr = expression.strip('"\'')
    return eval(expr)

# Initialize agent (direct tool assignment)
agent = initialize_agent(
    tools=[calculator],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # More robust parsing
    verbose=True,
    handle_parsing_errors=True  # Graceful error handling
)

response = agent.run("What is 15 * 4 + 2?")
print(response)  # Expected: "62"




> Entering new AgentExecutor chain...

<think>The human has asked me to solve a math problem: 15 * 4 + 2. This is a straightforward arithmetic expression that I can calculate directly.

Let me break this down:
1. First, multiply 15 by 4
2. Then add 2 to the result

I can use the calculator tool for this expression.

The expression is: "15 * 4 + 2"

I'll use the calculator tool to solve this.</think>Action:
```
{
  "action": "calculator",
  "action_input": {
    "expression": "15 * 4 + 2"
  }
}
```
Observation: 62
Thought:
<think>The calculator tool has computed that 15 * 4 + 2 equals 62. Let me verify this calculation manually:
- First, I multiply 15 by 4: 15 * 4 = 60
- Then I add 2 to the result: 60 + 2 = 62

The calculation is correct. Now I can provide the final answer to the human.</think>Action:
```
{
  "action": "Final Answer",
  "action_input": "The result of 15 * 4 + 2 is 62."
}
```

> Finished chain.
The result of 15 * 4 + 2 is 62.


**Which to choose?**

__Method 1 (Tool object):__ Better for complex tools requiring extra configuration

__Method 2 (@tool decorator):__ More concise and modern, recommended for simpler tools

In [ ]:
# Non-tool usage example

response = agent.run("Are Pandas is an endangered species or a Python library")
print(response)



> Entering new AgentExecutor chain...

<think>The human is asking whether "Pandas" refers to an endangered species or a Python library. This requires factual knowledge about both topics.

Pandas (the animal) are indeed an endangered species of bear native to China, known for their distinctive black and white coloring.

Pandas (the library) is also a very popular Python library used for data analysis and manipulation. It provides fast, flexible, and expressive data structures designed to make working with relational or labeled data both easy and intuitive.

The question is asking me to clarify which one they're referring to, but since both are valid interpretations of "Pandas," I should provide information about both concepts rather than making a definitive choice. This doesn't require the use of any calculator or tools, so I can respond directly with factual information about both pandas (the animal) and pandas (the Python library).</think>
```
{
  "action": "Final Answer",
  "action

### Example with more custom tools for LLM agent

__What is .invoke()?__
In LangChain, when you create a tool with @tool, it becomes a Tool object that has an .invoke() method instead of being called directly like a regular Python function.

This is because LangChain tools need to handle:

Input validation, Error handling, Logging, Integration with the agent system

In [71]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
from io import BytesIO
import os
from pathlib import Path

In [87]:
@tool
def browse_and_scrape(url: str) -> str:
    """
    Opens a webpage URL and extracts main text content.
    
    Args:
        url: The webpage URL to open (e.g., "https://example.com")
    
    Returns:
        Main text content of the webpage
    
    Example usage:
        browse_and_scrape("https://www.example.com")
    """
    try:
        # Set headers to mimic a browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove script and style elements
        for element in soup(["script", "style"]):
            element.decompose()
            
        # Get main text (focus on body content)
        if soup.body:
            text = ' '.join(soup.body.stripped_strings)
        else:
            text = ' '.join(soup.stripped_strings)
        print(f'Number of characters in text: {len(text)}')

        return f"Content from {url}:\n\n{text[:2000]}..."  # Truncate for brevity/lower token consumption
        
    except Exception as e:
        return f"Error accessing {url}: {str(e)}"


In [88]:
print(browse_and_scrape.invoke('https://en.wikipedia.org/wiki/Patulin'))

Number of characters in text: 14121
Content from https://en.wikipedia.org/wiki/Patulin:

Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Contents move to sidebar hide (Top) 1 Biosynthesis, synthesis, and reactivity 2 Uses 3 Sources of exposure 4 Toxicity Toggle Toxicity subsection 4.1 Acute 4.2 Subacute 4.3 Genotoxicity 4.4 Reproduction studies 4.5 Immunotoxicity 4.6 Human health 5 Risk management and regulations 6 References 7 External links Toggle the table of contents Patulin 23 languages العربية Català Čeština Deutsch Español Français 한국어 Italiano Latina Latviešu Magyar Nederlands 日本語 Polski Português Română Русский Српски / srpski Srpskohrvatski 

In [ ]:
@tool
def open_pdf(file_path: str) -> str:
    """
    Opens a PDF file specified by the user and extracts its text content.

    Args:
        file_path: Full path to the PDF file (e.g., "/path/to/document.pdf")
    
    Returns:
        Text content of the PDF file
    
    Example usage:
        open_pdf("/Users/user/documents/report.pdf")
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            return f"Error: File '{file_path}' does not exist"
            
        with open(file_path, 'rb') as f:
            # Create PDF reader
            pdf_reader = PyPDF2.PdfReader(f)
            
            # Extract text from each page
            text_parts = []
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text_parts.append(f"--- Page {page_num + 1} ---\n{page.extract_text()}")
                
            return f"PDF Content from {file_path}:\n\n" + "\n".join(text_parts)
            
    except Exception as e:
        return f"Error reading PDF '{file_path}': {str(e)}"


In [ ]:
print(open_pdf.invoke(('/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf')))

__Usage with LangChain LLM agent__

In [97]:
# Initialize your LLM (replace with your LM Studio config)
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    max_tokens=4096,
)

# Create agent with custom tools
agent = initialize_agent(
    tools=[browse_and_scrape, open_pdf],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# Example queries:
agent.run('''Summarize the following PDF document at path: /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf
           What are the main ideas behind the text? what is it about?
          ''')




> Entering new AgentExecutor chain...

<think>The human wants me to summarize a PDF document located at "/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf". They specifically want to know the main ideas and what the document is about.

To do this, I'll need to use the open_pdf tool to read and extract the text content from the specified PDF file. Then I'll analyze that content to provide a summary of the main ideas and purpose of the document.

Let me call the open_pdf tool with the provided file path.</think>
I'll help you summarize the PDF document. Let me first open and read the content of the file.
```
{
  "action": "open_pdf",
  "action_input": "/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf"
}
```
Observation: PDF Content from /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf:

--- Page 1 ---
Special Topic: Ag x BTO
BACKGROUND:
The Defense Advanced Research Projects Agency (DARPA) is looking for innovative 

"This document is a DARPA (Defense Advanced Research Projects Agency) solicitation notice for Special Topic: Ag x BTO (Agriculture × Biological Technologies Office). It's a call for proposals to develop innovative solutions for defending agriculture against both natural and man-made threats to enhance national security.\n\nMain ideas include:\n1. DARPA's Biological Technologies Office seeks to catalyze innovation in agricultural defense\n2. The solicitation focuses on five key technical areas:\n   - Early warning systems for chemical/biological threat surveillance\n   - Rapid-response agricultural countermeasures against threats\n   - Massively accelerated crop engineering for long-term solutions\n   - Integrated threat prediction and modeling systems\n   - Novel methods to detect human intervention and attribute provenance\n3. This is a limited submission period (June 1 - August 31, 2025)\n4. Selected submitters may be invited to attend a workshop and Pitch Day\n5. DARPA aims to encou

In [96]:
# Initialize your LLM (replace with your LM Studio config)
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    max_tokens=4096,
)

# Create agent with custom tools
agent = initialize_agent(
    tools=[browse_and_scrape, open_pdf],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example queries:
agent.run('''Open this URL page https://en.wikipedia.org/wiki/Patulin
           Browse and do basic scraping and find the IUPAC name of Patulin''')




> Entering new AgentExecutor chain...

<think>The human is asking me to open a Wikipedia page about patulin, scrape the content, and find the IUPAC name of patulin. Let me use the browse_and_scrape tool to access this URL and extract the content.

The URL is: https://en.wikipedia.org/wiki/Patulin

I'll use this tool to scrape the page and then look for the IUPAC name of patulin in the content.</think>Action:
```
{
  "action": "browse_and_scrape",
  "action_input": {
    "url": "https://en.wikipedia.org/wiki/Patulin"
  }
}
```Number of characters in text: 14121

Observation: Content from https://en.wikipedia.org/wiki/Patulin:

Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn

'The IUPAC name of Patulin is: 4-hydroxy-4H-furo[3,2-c]pyran-2(6H)-one'